In [1]:
import pandas as pd

In [3]:
# Загрузка данных
df1 = pd.read_excel('Магнит Уход и гигиена 07.24.0.xlsb', engine='pyxlsb')
df2 = pd.read_excel('Магнит Уход и гигиена 08.24.0.xlsb', engine='pyxlsb')

In [5]:
# Проверка идентичности таблиц на кол-во столбцов (В результате выполнения блока выводит ошибку, если порядок столбцов разный)
assert df1.columns.tolist() == df2.columns.tolist(), "Порядок столбцов в таблицах разный!"

AssertionError: Порядок столбцов в таблицах разный!

In [7]:
# Блок на проверку кол-ва столбцов, их названий и различий в обеих таблицах 
#(понадобился потому как предыдущий блок кода (Проверки идентичности таблиц) показал разный порядок колонок)
# Получаем названия столбцов
cols1 = df1.columns.tolist()
cols2 = df2.columns.tolist()

# Выводим базовую информацию
print("[Количество столбцов]")
print(f"Таблица 1: {len(cols1)}")
print(f"Таблица 2: {len(cols2)}\n")

print("[Названия столбцов]")
print(f"Таблица 1: {cols1}")
print(f"Таблица 2: {cols2}\n")

# Проверка на отличия
if cols1 == cols2:
    print("✅ Столбцы полностью идентичны (включая порядок)!")
else:
    print("⚠️ Обнаружены различия!")

    # Сравнение через множества (без учета порядка)
    set1 = set(cols1)
    set2 = set(cols2)

    if set1 == set2:
        print("-> Названия столбцов совпадают, но отличается порядок!")
    else:
        # Поиск уникальных столбцов
        unique_in_1 = list(set1 - set2)
        unique_in_2 = list(set2 - set1)
        
        if unique_in_1:
            print(f"-> Уникальные столбцы в Таблице 1: {unique_in_1}")
        if unique_in_2:
            print(f"-> Уникальные столбцы в Таблице 2: {unique_in_2}")

    # Проверка порядка
    if len(cols1) == len(cols2):
        order_diff = [i for i, (a, b) in enumerate(zip(cols1, cols2)) if a != b]
        if order_diff:
            print(f"-> Столбцы расположены в разном порядке. Несовпадения на позициях: {order_diff}")

[Количество столбцов]
Таблица 1: 23
Таблица 2: 24

[Названия столбцов]
Таблица 1: ['Месяц', 'Год', 'Код', 'Название магазина', 'Адрес', 'Основной РЦ', 'Регион', 'Формат', 'Субформат', 'Уровень', 'Уровень.1', 'Уровень.2', 'Уровень.3', 'Код позиции', 'Наименование', 'Бренд', 'Поставщик', 'Вес', 'Единица измерения', 'Штриховой код', 'Продажи в шт.', 'Себестоимсть в руб.', 'Продажи в руб.']
Таблица 2: ['Месяц', 'Год', 'Код', 'Название магазина', 'Адрес', 'Основной РЦ', 'Регион', 'Формат', 'Субформат', 'Уровень', 'Уровень.1', 'Уровень.2', 'Уровень.3', 'Уровень.4', 'Код позиции', 'Наименование', 'Бренд', 'Поставщик', 'Вес', 'Единица измерения', 'Штриховой код', 'Продажи в шт.', 'Себестоимсть в руб.', 'Продажи в руб.']

⚠️ Обнаружены различия!
-> Уникальные столбцы в Таблице 2: ['Уровень.4']


In [9]:
# Сохраняем исходный порядок столбцов из второй таблицы (так как в df2 на столбец больше)
original_columns = df2.columns.tolist()

In [11]:
# Объединение таблиц
combined_df = pd.concat([df1, df2])

In [13]:
# Определение столбцов для суммирования и группировки
sum_columns = ['Продажи в шт.', 'Себестоимсть в руб.', 'Продажи в руб.']
group_columns = ['Код', 'Код позиции']

In [15]:
# Создание словаря для агрегации
agg_dict = {col: 'sum' for col in sum_columns}
for col in combined_df.columns:
    if col not in sum_columns + group_columns:
        agg_dict[col] = 'first'

In [17]:
# Группировка и агрегация
result_df = combined_df.groupby(group_columns, as_index=False).agg(agg_dict)

In [19]:
# Проверка дубликатов
duplicates = result_df.duplicated(subset=['Код', 'Код позиции']).sum()
print(f"Найдено дубликатов: {duplicates}")

Найдено дубликатов: 0


In [21]:
# Проверка кол-ва строк после группировки (Необходимо для выбора формата сохранения файла)
print(f"Количество строк после группировки: {len(result_df)}")

Количество строк после группировки: 1323508


In [23]:
# Восстанавливаем исходный порядок столбцов
result_df = result_df[original_columns]

In [25]:
# Исходя из данных полученных в предыдущих двух блоках, разбиваем получившийся датафрейм на две части и сохраняем на двух ли
max_rows = 1_000_000  # Максимум для одного листа
chunks = [result_df[i:i+max_rows] for i in range(0, len(result_df), max_rows)]

with pd.ExcelWriter('Объединенный_отчет_Магнит_июль_август.xlsx', engine='openpyxl') as writer:
    for idx, chunk in enumerate(chunks):
        chunk.to_excel(writer, sheet_name=f'Лист_{idx+1}', index=False)